<a href="https://colab.research.google.com/github/Monukumarshekhar/Blogs/blob/main/Customer_Churn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas numpy scikit-learn xgboost shap streamlit joblib


In [2]:
from google.colab import files

uploaded = files.upload()  # Click "Choose Files" and select your CSV


Saving Telco-Customer-Churn.csv to Telco-Customer-Churn (1).csv


In [3]:
import pandas as pd

df = pd.read_csv('Telco-Customer-Churn.csv')
print(df.shape)  # Shows number of rows and columns
df.head()  # Display first 5 rows


(7043, 21)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [4]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [5]:
df.shape


(7043, 21)

In [6]:
df.head()


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [7]:
df['Churn'].value_counts()


,count
Churn,
No,5174
Yes,1869


In [8]:
df.describe()


,SeniorCitizen,tenure,MonthlyCharges
count,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692
std,0.368612,24.559481,30.090047
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.500000
50%,0.000000,29.000000,70.350000
75%,0.000000,55.000000,89.850000
max,1.000000,72.000000,118.750000


Data Cleaning and Preprocessing

In [9]:
# Convert 'TotalCharges' to a number, forcing errors (like empty spaces) into NaN (Not a Number)
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

# Fill any missing values with the median charge
df.dropna(inplace=True)

# Drop the customerID column as it's not needed for prediction
df.drop('customerID', axis=1, inplace=True)

# Now, convert all text columns into numbers using LabelEncoder
from sklearn.preprocessing import LabelEncoder

for column in df.columns:
  if df[column].dtype == 'object':
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])

print("Data successfully preprocessed!")
df.head()

Data successfully preprocessed!


,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,0,0,1,0,1,0,1,0,0,2,0,0,0,0,0,1,2,29.85,29.85,0
1,1,0,0,0,34,1,0,0,2,0,2,0,0,0,1,0,3,56.95,1889.50,0
2,1,0,0,0,2,1,0,0,2,2,0,0,0,0,0,1,3,53.85,108.15,1
3,1,0,0,0,45,0,1,0,2,0,2,2,0,0,1,0,0,42.30,1840.75,0
4,0,0,0,0,2,1,0,1,0,0,0,0,0,0,0,1,2,70.70,151.65,1


Model Training and Evaluation

In [10]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

# 1. Separate features (X) and the target variable (y)
X = df.drop('Churn', axis=1)
y = df['Churn']

# 2. Split data into training (80%) and testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Initialize and train the XGBoost model
# We use 'eval_metric' to prevent a warning message
model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)

# 4. Evaluate the model's performance on the test data
preds = model.predict_proba(X_test)[:, 1]
auc_score = roc_auc_score(y_test, preds)

print(f"Model trained successfully!")
print(f"ROC-AUC Score: {auc_score:.4f}")

Model trained successfully!
ROC-AUC Score: 0.8076


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [19:35:10] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Save Your Model

In [11]:
import joblib

# Save the model to a file named 'churn_model.pkl'
joblib.dump(model, 'churn_model.pkl')

print("Model saved successfully as churn_model.pkl")

Model saved successfully as churn_model.pkl


 Build and Launch Your Web App

In [13]:
%%writefile app.py
import streamlit as st
import pandas as pd
import joblib
from sklearn.preprocessing import LabelEncoder

# Load the trained model
model = joblib.load('churn_model.pkl')

# Set up the Streamlit page
st.set_page_config(page_title="Customer Churn Predictor", layout="wide")
st.title("Customer Churn Prediction 🔮")

# Create columns for inputs for a cleaner layout
col1, col2, col3 = st.columns(3)

with col1:
    gender = st.selectbox("Gender", ["Male", "Female"])
    senior_citizen = st.selectbox("Senior Citizen", ["No", "Yes"])
    partner = st.selectbox("Partner", ["No", "Yes"])
    dependents = st.selectbox("Dependents", ["No", "Yes"])

with col2:
    tenure = st.slider("Tenure (months)", 0, 72, 24)
    phone_service = st.selectbox("Phone Service", ["No", "Yes"])
    multiple_lines = st.selectbox("Multiple Lines", ["No", "Yes", "No phone service"])
    internet_service = st.selectbox("Internet Service", ["DSL", "Fiber optic", "No"])

with col3:
    monthly_charges = st.slider("Monthly Charges ($)", 18.0, 120.0, 70.0)
    total_charges = st.slider("Total Charges ($)", 18.0, 9000.0, 1400.0)
    contract = st.selectbox("Contract", ["Month-to-month", "One year", "Two year"])
    payment_method = st.selectbox("Payment Method", ["Electronic check", "Mailed check", "Bank transfer (automatic)", "Credit card (automatic)"])

# Create a dictionary of the inputs to pass to the model
input_data = {
    'gender': gender, 'SeniorCitizen': 1 if senior_citizen == 'Yes' else 0,
    'Partner': partner, 'Dependents': dependents, 'tenure': tenure,
    'PhoneService': phone_service, 'MultipleLines': multiple_lines,
    'InternetService': internet_service, 'OnlineSecurity': 'No', 'OnlineBackup': 'No',
    'DeviceProtection': 'No', 'TechSupport': 'No', 'StreamingTV': 'No',
    'StreamingMovies': 'No', 'Contract': contract, 'PaperlessBilling': 'Yes',
    'PaymentMethod': payment_method, 'MonthlyCharges': monthly_charges,
    'TotalCharges': float(total_charges)
}

# Convert the dictionary to a DataFrame
input_df = pd.DataFrame([input_data])

# Encode the categorical variables in the input DataFrame
for column in input_df.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    input_df[column] = le.fit_transform(input_df[column])

# Prediction button
if st.button("Predict Churn"):
    prediction_proba = model.predict_proba(input_df)[:, 1]
    churn_risk = prediction_proba[0]

    st.subheader("Prediction Result")
    if churn_risk > 0.5:
        st.error(f"High Churn Risk! Probability: {churn_risk:.2f}")
    else:
        st.success(f"Low Churn Risk. Probability: {churn_risk:.2f}")

Overwriting app.py


In [23]:
!pip install pandas scikit-learn xgboost joblib streamlit pyngrok -q

In [24]:
from pyngrok import ngrok

# PASTE YOUR NGROK AUTHTOKEN HERE
NGROK_AUTHTOKEN = "339xcAyr75KcQymW8JWcASuKTs1_5UAV4ABji6aYkqdAr881K"

# --- Do not edit below this line ---
ngrok.kill()
ngrok.set_auth_token(NGROK_AUTHTOKEN)
public_url = ngrok.connect(8501)
print(f"Click this link to view your app: {public_url}")

Click this link to view your app: NgrokTunnel: "https://unexceptionable-satisfiedly-edda.ngrok-free.dev" -> "http://localhost:8501"


In [25]:
!streamlit run app.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.106.250.134:8501



  Stopping...
  Stopping...
